<a href="https://colab.research.google.com/github/ihagoSantos/natural-language-processing/blob/main/text_representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instalação dos pacotes necessários
!pip install nltk==3.8.1
!pip install unidecode==1.3.8
!pip install scikit-learn==1.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 90.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [3]:
# Importação das bibliotecas padrão

import datetime
import os
import re
import string
import sys
import warnings

# Importação das bibliotecas de terceiros
import nltk
import pandas as pd

from nltk.corpus import stopwords

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from unidecode import unidecode

# Download do NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Desativando warnings
warnings.filterwarnings('ignore')

print("Pacotes importados com sucesso!")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Pacotes importados com sucesso!


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# Declara funções utilitárias
def formata_msg(nivel, msg):
  timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  return f'[{nivel}] {timestamp} - {msg}'

print(formata_msg("INFO", 'Funções utilitárias prontas para uso'))
print(formata_msg("INFO", f'Versão do python: {sys.version}'))

[INFO] 2024-12-05 09:34:46 - Funções utilitárias prontas para uso
[INFO] 2024-12-05 09:34:46 - Versão do python: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]


# Definição do Corpus

In [8]:
documentos =\
[
    # No meio do caminho - Carlos Drumond de Andrade
    'No meio do caminho tinha uma pedra\n'\
    'tinha uma pedra no meio do caminho\n'\
    'tinha uma pedra\n'\
    'no meio do caminho tinha uma pedra.\n'\
    'Nunca me esquecerei desse acontecimento\n'\
    'na vida de minhas reti nas tão fatigadas.\n'\
    'Nunca me esquecerei que no meio do caminho\n'\
    'tinha uma pedra\n'\
    'tinha uma pedra no meio do caminho\n'\
    'no meio do caminho tinha uma pedra.'
    ,
    # Quadrilha - Carlos Drumond de Andrade
    'João amava Teresa que amava Raimundo\n'\
    'que amava Maria que amava Joaquim que amava Lili,\n'\
    'que não amava ninguém.\n'\
    'João foi para os Estados Unidos, Teresa para o convento,\n'\
    'Raimundo morreu de desastre, Maria ficou pra titia,\n'\
    'Joaquim suicidou-se e Lili casou-se com J. Pinto Fernandes\n'\
    'que não tinha entrado na história.'
]

print(formata_msg('INFO', f"Documentos:\n{documentos}"))

[INFO] 2024-12-05 09:41:22 - Documentos:
['No meio do caminho tinha uma pedra\ntinha uma pedra no meio do caminho\ntinha uma pedra\nno meio do caminho tinha uma pedra.\nNunca me esquecerei desse acontecimento\nna vida de minhas reti nas tão fatigadas.\nNunca me esquecerei que no meio do caminho\ntinha uma pedra\ntinha uma pedra no meio do caminho\nno meio do caminho tinha uma pedra.', 'João amava Teresa que amava Raimundo\nque amava Maria que amava Joaquim que amava Lili,\nque não amava ninguém.\nJoão foi para os Estados Unidos, Teresa para o convento,\nRaimundo morreu de desastre, Maria ficou pra titia,\nJoaquim suicidou-se e Lili casou-se com J. Pinto Fernandes\nque não tinha entrado na história.']


# Pré-processamento

## Atividade
1) Escreva um método que realiza o pré-processamento da lista de documentos.\
O Método deve, para cada documento:
- Tokenizar cada palavra
- Remover stopwords
- Remover números
- Remover pontuações
- Remover acentos

In [34]:
def pre_processa_texto(texto):
  """
  Pré-processa o texto fornecido realizando várias etapas de limpeza.

  Etapas:
  1 - Tokeniza o texto
  2 - Converte os tokens para minúsculo
  3 - Remove stopwords em português
  4 - Remove números dos tokens
  5 - Exclui tokens que são pontuações
  6 - Remove acentuações dos tokens

  Parâmetros:
  texto (str): O texto a ser processado

  Retorna:
  lista: Lista de tokens pré-processados
  """
  padrao = r"\w+(?:'\w+)?|[^\w\s]"
  tokens_preprocessados = re.findall(padrao, texto)
  portugues_stops = stopwords.words('portuguese')
  resultado = []

  # Tokeniza o texto usando um padrão para capturar as palavras e pontuações
  padrao = r"\w+(?:'\w+)?|[^\w\s]"
  tokens_preprocessados = re.findall(padrao, texto)

  # Converte tokens para minusculos para padronizar a captalização
  tokens_preprocessados = [token.lower() for token in tokens_preprocessados]

  # Remove stopwords para reduzir o conjunto de tokens a palavras significativas
  portugues_stops = stopwords.words('portuguese')
  tokens_preprocessados = [token for token in tokens_preprocessados if token not in portugues_stops]

  # Remove números, pois geralmente não contribuem para o significado do texto
  tokens_preprocessados = [re.sub(r'\d+', '', token) for token in tokens_preprocessados if re.sub(r'\d+', '', token)]

  # Exclui tokens que são pontuações, pois raramente são úteis para a análise de texto.
  tokens_preprocessados = [token for token in tokens_preprocessados if token not in string.punctuation]

  # Remove acentuações para padronizar os tokens
  tokens_preprocessados = [unidecode(token) for token in tokens_preprocessados]

  return ' '.join(tokens_preprocessados)


In [35]:
documentos_preprocessados = [pre_processa_texto(poema) for poema in documentos]
print('INFO', f'Documentos preprocessados:\n{documentos_preprocessados}')

INFO Documentos preprocessados:
['meio caminho pedra pedra meio caminho pedra meio caminho pedra nunca esquecerei desse acontecimento vida reti tao fatigadas nunca esquecerei meio caminho pedra pedra meio caminho meio caminho pedra', 'joao amava teresa amava raimundo amava maria amava joaquim amava lili amava ninguem joao estados unidos teresa convento raimundo morreu desastre maria ficou pra titia joaquim suicidou lili casou j pinto fernandes entrado historia']


# Representação Textual

## Bag of Words

Faça um CountVectorizer nos documentos da variável documentos considerando binary = True

In [40]:
def aplica_bow(documentos_preprocessados):
  """
  Analisa documentos usando BoW e retorna o vocabulário extraído e um Dataframe contendo os valores do BoW para cada termo do documento.

  Parâmetros:
  - documentos_preprocessados: Lista de documentos pre-processados

  Retorna:
  - Um dicionário contendo o vocabulário extraído e o DataFrame dos valores do BoW.
  """

  # Inicia Vectorizer
  vectorizer = CountVectorizer(binary=True)

  # Transforma os documentos em uma matriz binária
  bow_matrix = vectorizer.fit_transform(documentos_preprocessados)

  # Obtem vocabulário extraído
  vocabulario = vectorizer.get_feature_names_out()

  # Cria um DataFrame para os valores do BoW
  df = pd.DataFrame(
      bow_matrix.T.todense(),
      index=vocabulario,
      columns = [f"doc{i+1}" for i in range(bow_matrix.shape[0])]
  )
  return {
      "vocabulario": vocabulario,
      "bow_dataframe": df
  }

In [41]:
bow_dict = aplica_bow(documentos_preprocessados)
print(formata_msg('INFO', f'Vocabulário BoW:\n{bow_dict["vocabulario"]}'))
print(formata_msg('INFO', f'DataFrame BoW:\n{bow_dict["bow_dataframe"]}'))

[INFO] 2024-12-05 10:14:35 - Vocabulário BoW:
['acontecimento' 'amava' 'caminho' 'casou' 'convento' 'desastre' 'desse'
 'entrado' 'esquecerei' 'estados' 'fatigadas' 'fernandes' 'ficou'
 'historia' 'joao' 'joaquim' 'lili' 'maria' 'meio' 'morreu' 'ninguem'
 'nunca' 'pedra' 'pinto' 'pra' 'raimundo' 'reti' 'suicidou' 'tao' 'teresa'
 'titia' 'unidos' 'vida']
[INFO] 2024-12-05 10:14:35 - DataFrame BoW:
               doc1  doc2
acontecimento     1     0
amava             0     1
caminho           1     0
casou             0     1
convento          0     1
desastre          0     1
desse             1     0
entrado           0     1
esquecerei        1     0
estados           0     1
fatigadas         1     0
fernandes         0     1
ficou             0     1
historia          0     1
joao              0     1
joaquim           0     1
lili              0     1
maria             0     1
meio              1     0
morreu            0     1
ninguem           0     1
nunca             1     0
pe

# TF-IDF
1) Faça o TF-IDF Vectorizer nos documentos sem alterar nenhum parâmetro

In [48]:
def aplica_tf_idf(documentos_preprocessados):
  """
  Analisa documentos usando TF-IDF e retorna o vocabulário extraído e um
  dataframe contendo os valores TF-IDF para cada termo do documento

  Parâmetros:
  - documentos_preprocessados: Lista de documentos pre-processados

  Retorna:
  - Um dicionário contendo o vocabulário extraído e o DataFrame dos valores do TF-IDF.
  """

  # Inicializa o TfidfVectorizer
  vectorizer = TfidfVectorizer()

  # Transforma documento em uma matriz TF-IDF
  tfidf_matrix = vectorizer.fit_transform(documentos_preprocessados)

  # Obtem vocabulário extraído
  vocabulario = vectorizer.get_feature_names_out()

  # Cria um DataFrame para os valores de TF-IDF
  df = pd.DataFrame(
      tfidf_matrix.T.todense(),
      index=vocabulario,
      columns = [f"doc{i+1}" for i in range(tfidf_matrix.shape[0])]
  )

  return {
      "vocabulario": vocabulario,
      "tfidf_dataframe": df
  }

In [49]:
tfidf_dict = aplica_tf_idf(documentos_preprocessados)
print(formata_msg('INFO', f'Vocabulário tfidf:\n{tfidf_dict["vocabulario"]}'))
print(formata_msg('INFO', f'DataFrame tfidf:\n{tfidf_dict["tfidf_dataframe"]}'))

[INFO] 2024-12-05 10:23:02 - Vocabulário tfidf:
['acontecimento' 'amava' 'caminho' 'casou' 'convento' 'desastre' 'desse'
 'entrado' 'esquecerei' 'estados' 'fatigadas' 'fernandes' 'ficou'
 'historia' 'joao' 'joaquim' 'lili' 'maria' 'meio' 'morreu' 'ninguem'
 'nunca' 'pedra' 'pinto' 'pra' 'raimundo' 'reti' 'suicidou' 'tao' 'teresa'
 'titia' 'unidos' 'vida']
[INFO] 2024-12-05 10:23:02 - DataFrame tfidf:
                   doc1     doc2
acontecimento  0.086066  0.00000
amava          0.000000  0.69282
caminho        0.516398  0.00000
casou          0.000000  0.11547
convento       0.000000  0.11547
desastre       0.000000  0.11547
desse          0.086066  0.00000
entrado        0.000000  0.11547
esquecerei     0.172133  0.00000
estados        0.000000  0.11547
fatigadas      0.086066  0.00000
fernandes      0.000000  0.11547
ficou          0.000000  0.11547
historia       0.000000  0.11547
joao           0.000000  0.23094
joaquim        0.000000  0.23094
lili           0.000000  0.23094
ma